In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "4a4a3e98",
   "metadata": {},
   "source": [
    "## Прогноз PCS (three-phase linear)\n",
    "\n",
    "Прогноз щотижневих продажів по SKU з додатковими регресорами."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "54a082eb",
   "metadata": {},
   "outputs": [],
   "source": [
    "from pathlib import Path\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "\n",
    "from three_phase_linear import ForecastConfig, run_three_phase_forecast\n",
    "\n",
    "DATA_PATH = Path('dataset_pcs.csv')\n",
    "OUTPUT_PATH = Path('pcs_three_phase_forecast.csv')\n",
    "GROUP_COLS = ['sku_id']\n",
    "TARGET_COLUMN = 'qty_total'\n",
    "REGRESSORS = [\n",
    "    'orders_qty', 'total_abc_numeric', 'avg_discount_perc_by_goods',\n",
    "    'max_discount_perc_by_goods', 'avg_goods_price_by_goods', 'oos__by_goods',\n",
    "    'war', 'covid', 'sin_quarter', 'cos_quarter', 'sin_month', 'cos_month',\n",
    "    'sin_week', 'cos_week'\n",
    "]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "5fb15beb",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Горизонт прогнозу: 134 тижнів\n"
     ]
    }
   ],
   "source": [
    "df = pd.read_csv(DATA_PATH)\n",
    "df['period'] = pd.to_datetime(df['period'])\n",
    "comma_cols = ['avg_discount_perc_by_goods', 'max_discount_perc_by_goods', 'avg_goods_price_by_goods', 'oos__by_goods', 'sin_month', 'cos_month', 'sin_week', 'cos_week']\n",
    "for col in comma_cols:\n",
    "    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)\n",
    "    df[col] = pd.to_numeric(df[col], errors='coerce')\n",
    "numeric_cols = ['qty_total', 'orders_qty', 'total_abc_numeric', 'war', 'covid', 'sin_quarter', 'cos_quarter']\n",
    "for col in numeric_cols:\n",
    "    df[col] = pd.to_numeric(df[col], errors='coerce')\n",
    "\n",
    "# Mark placeholder rows (future horizon) where target should be forecasted\n",
    "placeholder_mask = df['last_goods_sell_status'].isna() & df['oos__by_goods'].isna()\n",
    "df.loc[placeholder_mask, TARGET_COLUMN] = np.nan\n",
    "\n",
    "df = df.sort_values(GROUP_COLS + ['period']).reset_index(drop=True)\n",
    "\n",
    "forecast_horizon = int(df.loc[placeholder_mask, 'period'].nunique()) if placeholder_mask.any() else 0\n",
    "if forecast_horizon <= 0:\n",
    "    forecast_horizon = 4\n",
    "print(f'Горизонт прогнозу: {forecast_horizon} тижнів')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "3546ce93",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\base\\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.\n",
      "  self._init_dates(dates, freq)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.\n",
      "  warnings.warn(\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1380: RuntimeWarning: divide by zero encountered in log\n",
      "  aic = self.nobs * np.log(sse / self.nobs) + k * 2\n",
      "C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\statsmodels\\tsa\\holtwinters\\model.py:1387: RuntimeWarning: divide by zero encountered in log\n",
      "  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>group_key</th>\n",
       "      <th>train_rows</th>\n",
       "      <th>cv_mae</th>\n",
       "      <th>skipped_reason</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>598294111</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>598302302</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>599653937</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>599703217</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>746436995</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   group_key  train_rows  cv_mae skipped_reason\n",
       "0  598294111         119     NaN           None\n",
       "1  598302302         119     NaN           None\n",
       "2  599653937         119     NaN           None\n",
       "3  599703217         119     NaN           None\n",
       "4  746436995         119     NaN           None"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "input_cols = ['period', *GROUP_COLS, 'category_id', TARGET_COLUMN, *REGRESSORS]\n",
    "input_cols = list(dict.fromkeys(input_cols))\n",
    "config = ForecastConfig(\n",
    "    time_col='period',\n",
    "    target_col=TARGET_COLUMN,\n",
    "    group_cols=GROUP_COLS,\n",
    "    freq='W-MON',\n",
    "    forecast_horizon=forecast_horizon,\n",
    "    seasonal_periods=52,\n",
    "    min_history=20,\n",
    "    lags=(1, 2, 3, 4, 8, 12, 16),\n",
    "    rolling_windows=(3, 4, 8, 12),\n",
    "    additional_regressors=REGRESSORS,\n",
    "    random_search_iterations=0,\n",
    "    n_splits=3,\n",
    "    n_estimators=300,\n",
    "    target_transform=np.log1p,\n",
    "    target_inverse_transform=np.expm1,\n",
    "    random_state=46,\n",
    ")\n",
    "\n",
    "preds, summaries = run_three_phase_forecast(df[input_cols].copy(), config)\n",
    "preds = preds.rename(columns={\n",
    "    'prediction': 'qty_total_forecast',\n",
    "    f'{TARGET_COLUMN}_holtwinters': 'qty_total_baseline',\n",
    "})\n",
    "preds['qty_total_forecast'] = preds['qty_total_baseline']\n",
    "summary_report = pd.DataFrame({\n",
    "    'group_key': [s.group_key[0] for s in summaries],\n",
    "    'train_rows': [s.train_rows for s in summaries],\n",
    "    'cv_mae': [s.best_score for s in summaries],\n",
    "    'skipped_reason': [s.skipped_reason for s in summaries],\n",
    "})\n",
    "summary_report.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "f922633d",
   "metadata": {},
   "outputs": [
    {
     "ename": "NameError",
     "evalue": "name 'GROUP_COLS' is not defined",
     "output_type": "error",
     "traceback": [
      "\u001b[1;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[1;31mNameError\u001b[0m                                 Traceback (most recent call last)",
      "Cell \u001b[1;32mIn[1], line 1\u001b[0m\n\u001b[1;32m----> 1\u001b[0m merge_cols \u001b[38;5;241m=\u001b[39m [\u001b[38;5;241m*\u001b[39m\u001b[43mGROUP_COLS\u001b[49m, \u001b[38;5;124m'\u001b[39m\u001b[38;5;124mperiod\u001b[39m\u001b[38;5;124m'\u001b[39m]\n\u001b[0;32m      2\u001b[0m forecast_df \u001b[38;5;241m=\u001b[39m preds[merge_cols \u001b[38;5;241m+\u001b[39m [\u001b[38;5;124m'\u001b[39m\u001b[38;5;124mqty_total_forecast\u001b[39m\u001b[38;5;124m'\u001b[39m]]\u001b[38;5;241m.\u001b[39mcopy()\n\u001b[0;32m      3\u001b[0m static_map \u001b[38;5;241m=\u001b[39m df[[\u001b[38;5;124m'\u001b[39m\u001b[38;5;124msku_id\u001b[39m\u001b[38;5;124m'\u001b[39m, \u001b[38;5;124m'\u001b[39m\u001b[38;5;124mcategory_id\u001b[39m\u001b[38;5;124m'\u001b[39m]]\u001b[38;5;241m.\u001b[39mdrop_duplicates()\n",
      "\u001b[1;31mNameError\u001b[0m: name 'GROUP_COLS' is not defined"
     ]
    }
   ],
   "source": [
    "merge_cols = [*GROUP_COLS, 'period']\n",
    "forecast_df = preds[merge_cols + ['qty_total_forecast']].copy()\n",
    "static_map = df[['sku_id', 'category_id']].drop_duplicates()\n",
    "forecast_df = forecast_df.merge(static_map, on='sku_id', how='left')\n",
    "forecast_df = forecast_df.rename(columns={'qty_total_forecast': 'qty_total'})\n",
    "forecast_df = forecast_df.sort_values(GROUP_COLS + ['period']).reset_index(drop=True)\n",
    "forecast_df.to_csv(OUTPUT_PATH, index=False)\n",
    "\n",
    "forecast_df.tail()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "caef8c05",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>group_key</th>\n",
       "      <th>train_rows</th>\n",
       "      <th>cv_mae</th>\n",
       "      <th>skipped_reason</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>598294111</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>598302302</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>599653937</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>599703217</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>746436995</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>458</th>\n",
       "      <td>48609967412</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>459</th>\n",
       "      <td>48609984414</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>460</th>\n",
       "      <td>48788920432</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>461</th>\n",
       "      <td>48789681494</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>462</th>\n",
       "      <td>48789690935</td>\n",
       "      <td>119</td>\n",
       "      <td>NaN</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>463 rows × 4 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       group_key  train_rows  cv_mae skipped_reason\n",
       "0      598294111         119     NaN           None\n",
       "1      598302302         119     NaN           None\n",
       "2      599653937         119     NaN           None\n",
       "3      599703217         119     NaN           None\n",
       "4      746436995         119     NaN           None\n",
       "..           ...         ...     ...            ...\n",
       "458  48609967412         119     NaN           None\n",
       "459  48609984414         119     NaN           None\n",
       "460  48788920432         119     NaN           None\n",
       "461  48789681494         119     NaN           None\n",
       "462  48789690935         119     NaN           None\n",
       "\n",
       "[463 rows x 4 columns]"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "summary_report"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}